In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import pandas as pd 
import numpy as np
from sklearn.base  import BaseEstimator, TransformerMixin
from  sklearn.pipeline  import Pipeline
import pickle
from sklearn.metrics import accuracy_score
from  sklearn.multiclass import OneVsRestClassifier
import os
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier,PositiveNaiveBayesClassifier
import sys
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
#CLASS POUR CREER DES OBJECTS AVEC DES NOMS COMME ON VEUX
# s= S(hello="helloWorld",coucou="j'aime le poisson")
# s.hello ( "helloWorld)
# Change to dict 
# s.toDict()["hello"] ("helloWorld")
class S(object):
    def __init__(self, **entries): self.__dict__.update(entries)    
    def __str__(self):
        sf=[]
        for i,j in getParams(self).items():
            sf.append(str(i)+" "+str(j))
        return "\n".join(sf)
    def __repr__(self):
        return S.__str__(self)
    def __getitem__(self,e):
        return vars(self)[e]
    def toDict():
        return vars(self)
#USEFULL
def createVarLocalePython(k,v): vars()[k]=v
def createVarPython(k,v): globals()[k]=v
class ColumnSelector(object):
    """
    A feature selector for scikit-learn's Pipeline class that returns
    specified columns from a numpy array.

    """

    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, y=None):
        return X[self.cols]

    def fit(self, X, y=None):
        return self
class toHotCls(object):
    """
    A feature selector for scikit-learn's Pipeline class that returns
    specified columns from a numpy array.

    """

    def __init__(self, fn):
        self.fn = fn

    def transform(self, X, y=None):
        def toHot(arr,d):
            m=d
            k={"FALSE":-1}
            kk=[]
            for i,j in enumerate(m):
                k[j]=i
            for i in arr:
                kk.append(k[i])
            return np.array(kk,dtype=np.int)
        return toHot(X,self.fn)

    def fit(self, X, y=None):
        return self
class ToLabelYCls:
    def __init__(self, fn, delete=False):
        self.fn=fn
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        def GetTestY(d,fnc=fnc_basic):
            a=d
            aa=[]
            for i in a:
                s=False
                for k,v in fnc_basic.iteritems():
                    if v(i):
                        s=True
                        aa.append(k)
                if not s and not delete:
                    aa.append("PB")
            return aa
        return GetTestY(X,self.fn)
class RemoveIf:
    def __init__(self, fn,col=None):
        self.fn=fn
        self.col=col
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        XX=self.col if self.col is not None  else ":"
        return X[X[self.col].apply(lambda a: not self.fn(a))] if self.col is not None  else X[X.apply(lambda a: not self.fn(a))]
    
    

In [ ]:
#PRE-TRAITEMENT
import re, sys
import HTMLParser
from nltk.corpus import stopwords
h = HTMLParser.HTMLParser()
emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\uD83E[\uDD00-\uDDFF])|"
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83c[\udde0-\uddff])|"  # flags (iOS)
    u"([\u2934\u2935]\uFE0F?)|"
    u"([\u3030\u303D]\uFE0F?)|"
    u"([\u3297\u3299]\uFE0F?)|"
    u"([\u203C\u2049]\uFE0F?)|"
    u"([\u00A9\u00AE]\uFE0F?)|"
    u"([\u2122\u2139]\uFE0F?)|"
    u"(\uD83C\uDC04\uFE0F?)|"
    u"(\uD83C\uDCCF\uFE0F?)|"
    u"([\u0023\u002A\u0030-\u0039]\uFE0F?\u20E3)|"
    u"(\u24C2\uFE0F?|[\u2B05-\u2B07\u2B1B\u2B1C\u2B50\u2B55]\uFE0F?)|"
    u"([\u2600-\u26FF]\uFE0F?)|"
    u"([\u2700-\u27BF]\uFE0F?)"
    "+", flags=re.UNICODE) 

def remove_accent(s):
    import unicodedata
    s1 = unicode(s,'utf-8')
    return unicodedata.normalize('NFD', s1).encode('ascii', 'ignore') 

def remove_emoji(text):
    text = text.decode('utf8')
    return emoji_pattern.sub(r'', text).encode('utf8')

def removeBackslash(text):
    sk=re.compile(
    "\\\\n|\\\\r")
    return sk.sub(r'',text)
    
def traitement(text):
    return remove_accent(h.unescape(removeBackslash(remove_accent(remove_emoji(text))).lower()).encode("utf-8"))

def removeRegex(d,rec):
    return re.sub(rec, '',d)

def removeRegexList(d,arr):
    for i in arr:
        d=removeRegex(d,i)
    return d
def remoteMultiSpace(d):
    return " ".join(d.split())

class PreTraitement:
    @staticmethod
    def transform( X,name):
        return [remoteMultiSpace(traitement(removeRegex(str(i),r'(\'\s*\')|[0-9]'))) for i in X[name]]
s_wFr=stopwords.words('french')

In [ ]:
#TRAITEMENT TEXT NGRAM - LEMMA
from nltk.tokenize import RegexpTokenizer
import time
import treetaggerwrapper
tokenizerG = RegexpTokenizer(r'[\w\']+')

def getLemma(word):
    global memory
    s=time.time()
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr',TAGPARFILE="/Users/lucasiscovici/Downloads/lib/french-utf8.par")
    words2=[]
    words3={}
    for j,i in enumerate(word):
        if i in memory:
            words3[j]=memory[i]
        else:
            words2.append(i)
    if len(words2)>0:
        tags = tagger.tag_text(words2,tagonly=True)
    else:
        tags = []
    listla=[i.lemma for i in treetaggerwrapper.make_tags(tags)]
    listReturn =[]
    count=0
    for j,i in enumerate(word):
        if j in words3:
            listReturn.append(words3[j])
        else:
            listReturn.append(listla[count])
            memory[i]=listla[count]
            count+=1
    s2=time.time()
    return listReturn
def find_ngrams(input_list, n):
    return [" ".join(i) for i in zip(*[input_list[i:] for i in range(n)])]
def getBiGrams(inpute):
    return find_ngrams(inpute,2)
def getTriGrams(inpute):
    return find_ngrams(inpute,3)
def word_feats(words,tr=False):
    return dict([(word, True) for word in words]) if tr else [word for word in words]

In [ ]:
#LOAD DATA AND PRE-TRAIREMENT
def _loadData(delim="\|\|\|",resetIndex=True,DirData="dataTer/",names=["comment","rate","rateM"],debug=False):
    #FCT TO READ_CSV FROM FILENAME 
    importCsvFromFileName = lambda file_name : pd.read_csv(file_name,delimiter=delim,names=names)
    # Load AND Combine all Data Together
    def mergeAllData(Dir):
        data=pd.DataFrame()
        rootdir = Dir
        for subdir, dirs, files in os.walk(rootdir):
            print("Load Data MergeAllData: Dir: %s" % subdir)
            for file in files:
                data = data.append(importCsvFromFileName(os.path.join(subdir, file)))
        return data
    data = mergeAllData(DirData)
    #IF ResetIndex is True, reset Index  of dataFrame, from 0 to len(data)
    data = data.reset_index() if resetIndex else data
    if debug:
        print("Len Of Data: %s" % data.count())
    return data
def loadData(Reload=False,default_name="reviews",params_loadData=[]):
    Sreturn = lambda data,code: S(data=data,code=code)
    if os.path.isfile(default_name) and not Reload:
        return Sreturn(pd.read_pickle(default_name),0)
    return Sreturn(_loadData(*params_loadData),1)

def loadDataAndPreTraiIf(default_name="reviews",columnName="comment",paramsLoadData=[]):
    def applyPreTraitementTo(dest,source,column=columnName):
        dest[column]=PreTraitement.transform(source,column)
    #LOAD DATA - DEJA PRE-TRAITÉES
    def saveReviewsPreTraited(d,name):   
        d.to_pickle(name)
    Sdata=loadData(*paramsLoadData)
    reviews=data=Sdata.data
    if Sdata.code==1:
        applyPreTraitementTo(reviews,data,columnName)
        saveReviewsPreTraited(reviews,default_name)
    return reviews

#LOAD MEMORY FOR TREE TAGER CALL
def memoryFC():
    memory=None
    def loadMemory():
        global memory
        memory=np.load("memory").tolist()

    #SAVE
    def saveMemory():
        global memory
        np.array(memory).dump("memory")

In [ ]:
#CopyData
reviews=loadDataAndPreTraiIf()

In [ ]:

#Pre-Traitement pour preparer les data pour le training format en dictionnaire
#UNIGRAM -> lemminization
#sentence to Word and Apply Funct Fn dessus
def buildToWord(nltkFormat=True):
    _fcDef= lambda j:j # FCP PAR DEFAUT (FAIT RIEN)
    __toWord = lambda fn, i, jj=False : fn([unicode(j,'utf-8') for j in tokenizerG.tokenize(i) ],jj) #BASE POUR TOKENISER
    _wf=lambda fn: lambda i,jj=False: word_feats(fn(i),jj) #BASE POUR FORMAT NLTK 

    _toWord = lambda fc: lambda i, jj=False : __toWord(_wf(fc),i,jj)
    _toWordBis=lambda k: lambda n: words[k](n,nltkFormat)
    baseWord="toWord"
    Words={"WithoutLemma":_fcDef,"Uni":getLemma,"Bi":getBiGrams,"Tri":getTriGrams}
    words={}
    wordsBi={}
    for k,v in Words.iteritems():
        #words+=[S(toWord=_toWord(v),name=baseWord+k)]
        words[k]=_toWord(v)
        wordsBi[k]=_toWordBis(k)
    return (words,wordsBi)

In [ ]:
toWORDS=buildToWord()

In [ ]:

#apply fct on data 
# For Filter
# Per exemple {"neg":lambda d: d[d.rate < 3.]}

#CREATE ARRAY WITH APLICATION OF len(FNS) FUNCTIONS ON DATA
def prepareData(data,fns):
    return [v(data) for k,v in fns.iteritems()]

#BUILD CLASSIFIER 
# dataset
# fc: function to apply for this classifer to build dataSet (create n-gram)
# debug
# sklearn : Not use nltk Naive Bayes, But the Naive Bayes of Sklearn (But with the help of nltk)
# sklearn : NaiveBayes to Use pour Sklearn (BernoulliNB, GaussianNB, MultinomialNB)
# fnc : How to prepareData ?  Which Data To Select ? Wich Label ? {"neg":lambda d: d[d.rate < 3.]}
# testPart: Percentage of train Data
# CollunsName To select for the dataset (comment)
def iiOk(d,maxx):
    print(d,maxx)
    return [0]*(d)+[1]+[0]*(maxx-d-1)

#
def get_classifier(dataset=reviews2,fc=toWord,debug=False,sklearn=False,sklearnNB=BernoulliNB,
                   fnc={},testPart=3/4.,colunmName="comment",arrLabelFormat=False):
        
    arr=prepareData(dataset,fnc)#Prepare Data
    print("Len: "+str(len(dataset)))
    #print(arr)
    Data=[]
    #For each Key in fnc (each Label , each Y)
    for i,k in enumerate(fnc):
        if debug:
            print(k+":")
            sys.stdout.flush()
        data=arr[i][colunmName] #Get prepared Data # LES COMMENTAIRES
        dataUp=[]
        for ii,f in enumerate(data):  #Loop Data COMMENTS
            if debug:
                sys.stdout.write(str(ii/float(len(data)))+"\r")
                sys.stdout.flush()
            dataUp.append((fc(f), [k] if arrLabelFormat else k) ) #Build n-gram with function fc
        
        Data.append(dataUp)
        if debug:
            sys.stdout.flush()
            sys.stdout.write("\r1     \n")
            sys.stdout.flush()
    
    DataTrainTest=[]
    for i in Data: #Know what size of each Label We Want 
        #print(len(i))
        DataTrainTest.append(int(len(i)*testPart))
    
    #Build train ANd Test DataSet
    trainfeats = []
    testfeats  = []
    for i,j in enumerate(Data):
        #print(DataTrainTest[i])
        trainfeats+=j[:DataTrainTest[i]]
        testfeats+=j[DataTrainTest[i]:]
    if debug:
        print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))
        print(trainfeats)
    
    classifier = SklearnClassifier(sklearnNB()).train(trainfeats) if sklearn else NaiveBayesClassifier.train(trainfeats)
    k=nltk.classify.util.accuracy(classifier, testfeats)
    if debug:
        print 'accuracy:', k
        if not sklearn: classifier.show_most_informative_features()
    return [[classifier,k]]

In [ ]:
#SI get_classifier for more explanation
nbl=0 #Lower Bound
nb=1000 # Upper Bound Test sur nb val
debug=False
Sklearn=True
SklearnParamNB=MultinomialNB
reviews2=reviews[nbl:nb]
toWordsBis=toWORDS[1]
testVar={
    "_1gram":toWordsBis["Uni"],
    "_2gram":toWordsBis["Bi"],
    "_3gram":toWordsBis["Tri"]
} # POUR AJOUTER/SUPPR/MODIF DES CLASSIFIEURS c'est ICI

fnc_basic={
    "neg":lambda d: d < 3.,
    "pos":lambda d: d > 3
} # BASE FOR FIND LABELS,POUR ENLEVER OU AJOUTER DES LABELS C'esT ICI
select= lambda d,e,f :d[d.rate.apply(lambda dd:f(dd))]
fnc={k:(lambda v: lambda d:select(d,"rate",v))(v) for k,v in fnc_basic.items()} # BASE FOR FIND IN DATAFRAM

In [ ]:
uni=(1,1)
bi=(2,2)
tri=(3,3)
XpipeParams={"UNI":uni,"BI":bi,"TRI":tri}

In [692]:
%time
def getPipelinesX(arr,fit_transform=None):
    def getPipelineX(ngram=(1,1),max_df=0.5,min_df=2,fit_transform=None):
        
        vect=('vect', CountVectorizer(ngram_range=ngram,max_df=max_df,min_df=min_df))
        vecto=('vecto', CountVectorizer(ngram_range=ngram))
        colS=("colSel",ColumnSelector("comment"))
        normal=[colS,
            vect,
               ('clf', MultinomialNB()) ]
        rmIf=("removeIf",RemoveIf(lambda d: d == 3.,"rate"))
        nnormal=normal
        
        ke=Pipeline(nnormal)
        if fit_transform is not None:
            ke.fit_transform(fit_transform)
        
        return [ke,Pipeline([rmIf])]
    return {k:getPipelineX(ngram=v,min_df=3,max_df=.5) for k,v in arr.iteritems()}
p=getPipelinesX(XpipeParams)

#features_nd = features.toarray()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 35 µs


In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(p["UNI"][-1].fit_transform(reviews2),labels)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression().fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print 'accuracy score: %s' % '{:.2%}'.format(score)
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print 'null accuracy: %s' % '{:.2%}'.format(null_accuracy)
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print 'model is %s more accurate than null accuracy' % '{:.2%}'.format(difference)
    elif difference < 0:
        print 'model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference))
    elif difference == 0:
        print 'model is exactly as accurate as null accuracy'
    return null_accuracy, accuracy_score

def train_test_and_evaluate(pipeline, X_train, y_train, X_test, y_test):
    pipeline.fit(X_train, y_train)
    y_pred_class = pipeline.predict(X_test)
    display_accuracy_difference(y_test, y_pred_class)
    classification_reports = confusion_matrix(y_test, y_pred_class)
    print '-' * 75 + '\nConfusion Matrix\n'
    print classification_reports
    print '-' * 75 + '\nClassification Report\n'
    print classification_report(y_test, y_pred_class)
      
    return pipeline, confusion_matrix

In [ ]:
#from sklearn_helpers import train_test_and_evaluate
sentiment_pipeline, confusion_matrix = train_test_and_evaluate(LogisticRegression(), X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.ensemble import VotingClassifier
xx=[]
for k,v in p.iteritems():
    xx+=[(k,v[0])]
v=VotingClassifier(estimators=xx,voting="soft")

In [ ]:
v.fit(X_train,y_train)

In [ ]:
sentiment_pipeline, confusion_matrix = train_test_and_evaluate(v.estimators_[1], X_train, y_train, X_test, y_test)

In [693]:
def test(debut=1000,fin=100000,pas=1000,voting="soft"):
    global reviews,XpipeParams,fnc_basic
    from matplotlib.legend_handler import HandlerLine2D
    p2=Pipeline([
        ("removeIf",RemoveIf(lambda d: d == 3.,"rate")),
        ("colSel",ColumnSelector("rate")),
        ('ToLabelY', ToLabelYCls(fnc_basic,True))
        ])
    absice=list(range(debut,fin,pas))
    ordonne={i:[] for i in range(6)}
    #print(ordonne)
    vvv=[]
    plt.figure()
    plt.title("Comparaison NB(nGRams et Voting(Soft|Hard)+stacking)")
    plt.xlabel("Training examples")
    plt.ylabel("Accuracy")
    p=getPipelinesX(XpipeParams)
    for i in range(debut,fin,pas):
        reviews2=reviews[:i]
        #print(i)
        labels=p2.fit_transform(reviews2)
        X_train, X_test, y_train, y_test  = train_test_split(p["UNI"][-1].fit_transform(reviews2),labels)
        from sklearn.ensemble import VotingClassifier
        xx=[]
        for k,v in p.iteritems():
            xx+=[(k,v[0])]
        #print(i)
        sclf = StackingClassifier(classifiers=[i[1] for i in xx],
                          use_probas=True,
                          average_probas=False,
                          meta_classifier=LogisticRegression())
        sclf.fit(X_train,y_train)
        vv=VotingClassifier(estimators=xx,voting=voting)
        vv.fit(X_train,y_train)
        vv2=VotingClassifier(estimators=xx,voting="hard")
        vv2.fit(X_train,y_train)
        ordonne[0].append(accuracy_score(y_test,vv.predict(X_test)))
        ordonne[1].append(accuracy_score(y_test,vv2.predict(X_test)))
        ordonne[2].append(accuracy_score(y_test,sclf.predict(X_test)))

        for i in range(len(vv.estimators_)):
            #print"ii:"+str(i)
            #print("n: "+str(i+1))
            ordonne[i+3].append(accuracy_score(toHotCls(fnc_basic).transform(y_test),vv.estimators_[i].predict(X_test)))
        vvv.append(vv)
    #print(ordonne)
    for z,(pd,i) in enumerate(ordonne.iteritems()):
        #print(absice)
        #print(i)
        line1,=plot(absice,i,label=vvv[0].estimators[z-3][0] if z>2 else ("Stacking" if z>1 else ("VotingHard" if z>0 else "VotingSoft")))
    #plt.legend(handler_map={line1: HandlerLine2D(numpoints=4)})
    plt.legend(bbox_to_anchor=(1, 0.35), loc=0, borderaxespad=0.)
    show()
    return vvv

In [697]:
ab=test(0,100000,20000)

KeyError: 'rate'

In [695]:
ab

[VotingClassifier(estimators=[('TRI', Pipeline(steps=[('colSel', <__main__.ColumnSelector object at 0x122b87950>), ('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
         dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
         lowercase=True, max_df=1.0, max_features=...zer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]))],
          n_jobs=1, voting='soft', weights=None),
 VotingClassifier(estimators=[('TRI', Pipeline(steps=[('colSel', <__main__.ColumnSelector object at 0x122b87950>), ('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
         dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
         lowercase=True, max_df=1.0, max_features=...zer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]))],
          n_jobs=1, voting='soft', weights=None),
 VotingClassifier(estimators=[('TRI', Pipeline(ste

In [687]:
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
xx=[]
for k,v in p.iteritems():
    xx+=[v[0]]
sclf = StackingClassifier(classifiers=xx,
                          use_probas=True,
                          average_probas=False,
                          meta_classifier=LogisticRegression())

In [688]:
sclf.fit(X_train,y_train)

StackingClassifier(average_probas=False,
          classifiers=[Pipeline(steps=[('colSel', <__main__.ColumnSelector object at 0x11ce348d0>), ('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, m...izer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])],
          meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=True, verbose=0)

In [689]:
sclf.score(X_test,y_test)

0.85046728971962615

In [ ]:
#BUILD Var In Key of testVar with as value the classifier
for k,v in testVar.iteritems():
    cal=get_classifier(reviews2Tested,v,debug,Sklearn,SklearnParamNB,fnc=fnc)
    val=[DecoClassif(cal[i][0],v,name=str(k)+"_"+fnc_basic.keys()[i]) for i in range(len(cal))] # FOR USE SKLEARN WE BUILD AN DECORATOR ( WITH PREDICT AND PREDICT_PROBA)
    createVarPython(k,[val,cal[i][1]])

In [ ]:
import codecs, re, time
from itertools import chain
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB



# Vectorizing data.
#train = []
word_vectorizer = _3gram[0].decore._vectorizer
#trainset = word_vectorizer.fit_transform(codecs.open(trainfile,'r','utf8'))
#tags = ['bs','pt','bs','pt']

# Training NB
mnb = _3gram[0].decore._clf
#mnb.fit(trainset, tags)

def most_informative_feature_for_binary_classification(vectorizer, classifier, n=100):
    class_labels = classifier.classes_
    feature_names = vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.coef_[0], feature_names))[:n]
    topn_class2 = sorted(zip(classifier.coef_[0], feature_names))[-n:]

    for coef, feat in topn_class1:
        print class_labels[0], coef, feat

    print

    for coef, feat in reversed(topn_class2):
        print class_labels[1], coef, feat


most_informative_feature_for_binary_classification(word_vectorizer, mnb)

In [ ]:
#To Test
begin=100
end=1000
step=100
for i in range(begin,end,step):
    print("i: "+str(i))
    for j,v in testVar.iteritems():
        print(j)
        print("\t"+str(vars()[j][1]))

In [ ]:
#WE CREATE AN VOTING CLASSIFIER THAT NOT NEED TO FIT DATA | ONLY PREDICT or PREDICT_DATA
from sklearn.ensemble import VotingClassifier
Estimators=[("n"+str(i+1),vars()[j][0]) for i,j in enumerate(testVar)]
eclf1 = VotingClassifier2(estimators=
                          Estimators, 
                          voting='soft',y=fnc.keys()
                         )

In [ ]:
#ACCURACY
 
def accuracy_scoreWithPredict(X,Y,classif,Hot=True):
    print(accuracy_score(GetTestY(X),classif.predict(Y,Hot=Hot)))

def accuracy_scoreWithPredictProba(X,Y,classif):
    print(accuracy_score(GetTestY(X),classif.predict(Y)))
    
#accuracy_score(GetTestY(reviews2[500:900]["rate"]),
#               toLabel([np.argmax(i) for i in eclf1.predict_proba(reviews2[500:900]["comment"])
 #              ],Uni.decore)
#             )

In [ ]:
accuracy_scoreWithPredict(reviews2[5000:6000]["rate"],reviews2[5000:6000]["comment"],_3gram[0],False)

In [ ]:
%time accuracy_scoreWithPredict(reviews2[50000:100000]["rate"],reviews2[50000:100000]["comment"],_2gram[0],False)

In [ ]:
#To Test
begin=100
end=1000
step=100
for i in range(begin,end,step):
    print("i: "+str(i))
    for j,v in testVar.iteritems():
        print(j)
        print("\t"+str(vars()[j][1]))

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)
print("Number of mislabeled points out of a total %d points : %d"
      % (iris.data.shape[0],(iris.target != y_pred).sum()))

In [ ]:
print(metrics.classification_report(y_test, prob))

In [ ]:
print(metrics.classification_report(y_test, prob))
print(metrics.confusion_matrix(iris.target, y_pred))
print(metrics.accuracy_score(iris.target, y_pred))
print(metrics.adjusted_mutual_info_score(iris.target, y_pred))

In [ ]:
print(metrics.accuracy_score(log_model.predict(X_test), y_test))

In [ ]:
print(metrics.cluster.completeness_score(iris.target, y_pred))

In [ ]:
print(metrics.cluster.v_measure_score(iris.target, y_pred))

In [ ]:


import numpy as np
import warnings

from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin
from sklearn.preprocessing import LabelEncoder
LabelEncoder
class VotingClassifier2( ClassifierMixin, TransformerMixin):


    def __init__(self, estimators, voting='hard', weights=None, n_jobs=1,
                 flatten_transform=None,y=[]):
        self.estimators = estimators
        self.estimators_ = [ i[1] for i in estimators]
        self.le_ = LabelEncoder().fit(y)
        self.voting = voting
        self.weights = weights
        self.n_jobs = n_jobs
        self.flatten_transform = flatten_transform

    @property
    def named_estimators(self):
        return dict(self.estimators)

    def fit(self, X, y, sample_weight=None):
        return self

    @property
    def _weights_not_none(self):
        """Get the weights of not `None` estimators"""
        if self.weights is None:
            return None
        return [w for est, w in zip(self.estimators,
                                    self.weights) if est[1] is not None]

    def predict(self, X,Hot=True):
        """ Predict class labels for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        maj : array-like, shape = [n_samples]
            Predicted class labels.
        """

        #check_is_fitted(self, 'estimators_')
        if self.voting == 'soft':
            maj = np.argmax(self.predict_proba(X), axis=1)

        else:  # 'hard' voting
            predictions = self._predict(X,Hot=Hot)
            print(predictions)
            maj = np.apply_along_axis(
                lambda x: np.argmax(
                    np.bincount(x, weights=self._weights_not_none)),
                axis=1, arr=predictions)

        maj = self.le_.inverse_transform(maj)

        return maj

    def _collect_probas(self, X):
        """Collect results from clf.predict calls. """
        return np.asarray([clf.predict_proba(X) for clf in self.estimators_])

    def _predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting """
        if self.voting == 'hard':
            raise AttributeError("predict_proba is not available when"
                                 " voting=%r" % self.voting)
        #check_is_fitted(self, 'estimators_')
        avg = np.average(self._collect_probas(X), axis=0,
                         weights=self._weights_not_none)
        return avg

    @property
    def predict_proba(self):
        """Compute probabilities of possible outcomes for samples in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        return self._predict_proba

    def transform(self, X):
        """Return class labels or probabilities for X for each estimator.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        -------
        If `voting='soft'` and `flatten_transform=True`:
          array-like = (n_classifiers, n_samples * n_classes)
          otherwise array-like = (n_classifiers, n_samples, n_classes)
            Class probabilities calculated by each classifier.
        If `voting='hard'`:
          array-like = [n_samples, n_classifiers]
            Class labels predicted by each classifier.
        """
        #check_is_fitted(self, 'estimators_')

        if self.voting == 'soft':
            probas = self._collect_probas(X)
            if self.flatten_transform is None:
                warnings.warn("'flatten_transform' default value will be "
                              "changed to True in 0.21."
                              "To silence this warning you may"
                              " explicitly set flatten_transform=False.",
                              DeprecationWarning)
                return probas
            elif not self.flatten_transform:
                return probas
            else:
                return np.hstack(probas)

        else:
            return self._predict(X)

    def set_params(self, **params):
        """ Setting the parameters for the voting classifier
        Valid parameter keys can be listed with get_params().
        Parameters
        ----------
        params: keyword arguments
            Specific parameters using e.g. set_params(parameter_name=new_value)
            In addition, to setting the parameters of the ``VotingClassifier``,
            the individual classifiers of the ``VotingClassifier`` can also be
            set or replaced by setting them to None.
        Examples
        --------
        # In this example, the RandomForestClassifier is removed
        clf1 = LogisticRegression()
        clf2 = RandomForestClassifier()
        eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)])
        eclf.set_params(rf=None)
        """
        super(VotingClassifier, self)._set_params('estimators', **params)
        return self

    def get_params(self, deep=True):
        """ Get the parameters of the VotingClassifier
        Parameters
        ----------
        deep: bool
            Setting it to True gets the various classifiers and the parameters
            of the classifiers as well
        """
        return super(VotingClassifier,
                     self)._get_params('estimators', deep=deep)

    def _predict(self, X,Hot=False):
        """Collect results from clf.predict calls. """
        #print([clf.predict(X) for clf in self.estimators_])
        return np.asarray([ toHotCls(fnc_basic).transform(clf.predict(X)) if Hot else clf.predict(X)  for clf in self.estimators_]).T

In [ ]:

from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin

class DecoClassifX( ClassifierMixin, TransformerMixin,BaseEstimator):

    
    def __init__(self,decore=None,fctFit=None, fctPredict=None,name="test"):
        self.decore = decore
        self.fn = fctPredict
        self.fnFit=fctFit
        print(self.fnFit)
        self.n=fctFit
        print(fctFit)
        self.name=name
    def setUP(self,decore,fctFit, fctPredict,name="test"):
        self.decore = decore
        self.fn = fctPredict
        self.fnFit=fctFit
        print(self.fnFit)
        self.n=fctFit
        print(fctFit)
        self.name=name
        return self
       
    def get_params(self, deep=True):
        """ Get the parameters of the VotingClassifier
        Parameters
        ----------
        deep: bool
            Setting it to True gets the various classifiers and the parameters
            of the classifiers as well
        """
        from copy import deepcopy
        au= super(DecoClassifX,
                     self).get_params(deep=deep)
        return deepcopy(self)
    def fit(self, X, y, sample_weight=None):
        print(self.fnFit)
        print(self.n)
        return self.decore.fit(self.fnFit(X),y)
    
    def predict(self, X):
        """ Predict class labels for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        maj : array-like, shape = [n_samples]
            Predicted class labels.
        """
        
        print(self.fn(X))
        k=self.decore.predict(self.fnFit(X))
        return  k

    def _collect_probas(self, X):
        """Collect results from clf.predict calls. """
        return self.decore._collect_probas(X)

    def _predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting """
        #check_is_fitted(self, 'estimators_')
        XX=[]
        for i,f in enumerate(X):
            XX.append(self.fn(f))
        return self.decore.predict_proba(X)

    @property
    def predict_proba(self):
        """Compute probabilities of possible outcomes for samples in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        return self._predict_proba

    def transform(self, X):
        """Return class labels or probabilities for X for each estimator.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        -------
        If `voting='soft'` and `flatten_transform=True`:
          array-like = (n_classifiers, n_samples * n_classes)
          otherwise array-like = (n_classifiers, n_samples, n_classes)
            Class probabilities calculated by each classifier.
        If `voting='hard'`:
          array-like = [n_samples, n_classifiers]
            Class labels predicted by each classifier.
        """
        return self.decore.transform(X)

    def _predict(self, X):
        """Collect results from clf.predict calls. """
        return self.decore._predict(X)
    def __repr__(self):
        return "DecoClassifX: "+self.name

In [ ]:

from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin

class DecoClassif( ClassifierMixin, TransformerMixin):


    def __init__(self, decore, fctPredict,name="test"):
        self.decore = decore
        self.fn = fctPredict
        self.name=name
       

    @property
    def named_estimators(self):
        return self.decore.named_estimators

    def fit(self, X, y, sample_weight=None):
        return self

    @property
    def _weights_not_none(self):
        """Get the weights of not `None` estimators"""
        return self.decore._weights_not_none

    def predict(self, X,Hot=True):
        """ Predict class labels for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        maj : array-like, shape = [n_samples]
            Predicted class labels.
        """
        XX=[]
        for i,f in enumerate(X):
            XX.append(self.fn(f))
        k=self.decore.classify_many(XX)
        return  toHot(k,self.decore.labels()) if Hot else k

    def _collect_probas(self, X):
        """Collect results from clf.predict calls. """
        return self.decore._collect_probas(X)

    def _predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting """
        #check_is_fitted(self, 'estimators_')
        XX=[]
        for i,f in enumerate(X):
            XX.append(self.fn(f))
        return giveMeProba(self.decore.labels(),self.decore.prob_classify_many(XX))

    @property
    def predict_proba(self):
        """Compute probabilities of possible outcomes for samples in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        return self._predict_proba

    def transform(self, X):
        """Return class labels or probabilities for X for each estimator.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        -------
        If `voting='soft'` and `flatten_transform=True`:
          array-like = (n_classifiers, n_samples * n_classes)
          otherwise array-like = (n_classifiers, n_samples, n_classes)
            Class probabilities calculated by each classifier.
        If `voting='hard'`:
          array-like = [n_samples, n_classifiers]
            Class labels predicted by each classifier.
        """
        return self.decore.transform(X)


    def set_params(self, **params):
        """ Setting the parameters for the voting classifier
        Valid parameter keys can be listed with get_params().
        Parameters
        ----------
        params: keyword arguments
            Specific parameters using e.g. set_params(parameter_name=new_value)
            In addition, to setting the parameters of the ``VotingClassifier``,
            the individual classifiers of the ``VotingClassifier`` can also be
            set or replaced by setting them to None.
        Examples
        --------
        # In this example, the RandomForestClassifier is removed
        clf1 = LogisticRegression()
        clf2 = RandomForestClassifier()
        eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)]
        eclf.set_params(rf=None)
        """
        super(VotingClassifier, self)._set_params('estimators', **params)
        return self

    def get_params(self, deep=True):
        """ Get the parameters of the VotingClassifier
        Parameters
        ----------
        deep: bool
            Setting it to True gets the various classifiers and the parameters
            of the classifiers as well
        """
        return super(VotingClassifier,
                     self)._get_params('estimators', deep=deep)

    def _predict(self, X):
        """Collect results from clf.predict calls. """
        return self._predict(X)
    def __repr__(self):
        return "DecoClassif: "+self.name

In [ ]:
def giveMeProba(Bi,arr):
    m=Bi
    k=[]
    for i in arr:
        o=[]
        for j in m:
            o.append(i.prob(j))
        k.append(o)
    return k
def toHot(arr,d):
    m=d
    k={"FALSE":-1}
    kk=[]
    for i,j in enumerate(m):
        k[j]=i
    for i in arr:
        kk.append(k[i])
    return np.array(kk,dtype=np.int)
def toLabel(arr,d):
    m=d
    k={}
    kk=[]
    for i,j in enumerate(m):
        k[i]=j
    for i in arr:
        kk.append(k[i])
    return np.array(kk,dtype=np.str)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples=1000, n_features=4,
                                 n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

In [ ]:
a=RandomForestClassifier()

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt



In [ ]:
plot_learning_curve(v.estimators_[0],"Voting",X_test,y_test)